In [504]:
import pandas as pd

round_path_file = "../../data/rounds_table.csv"

df = pd.read_csv(round_path_file, encoding='utf_16')

# list of columns selected for preprocessing
# columns deselected: 'url_transmissao', 'transmissao', 'url_confronto',
round_col_select = ['aproveitamento_mandante', 'aproveitamento_visitante', 'clube_casa_id', 'clube_casa_posicao', 
                  'clube_visitante_id', 'clube_visitante_posicao', 'local', 'partida_data', 'partida_id', 
                  'placar_oficial_mandante', 'placar_oficial_visitante', 'rodada_id',  'valida']

df = df[round_col_select]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 13 columns):
aproveitamento_mandante     280 non-null object
aproveitamento_visitante    280 non-null object
clube_casa_id               280 non-null int64
clube_casa_posicao          280 non-null int64
clube_visitante_id          280 non-null int64
clube_visitante_posicao     280 non-null int64
local                       280 non-null object
partida_data                280 non-null object
partida_id                  280 non-null int64
placar_oficial_mandante     277 non-null float64
placar_oficial_visitante    277 non-null float64
rodada_id                   280 non-null int64
valida                      280 non-null bool
dtypes: bool(1), float64(2), int64(6), object(4)
memory usage: 26.6+ KB


In [505]:
# counts the n occurrances of result in a list in a column in df
def count_perform(df, column, result, n):
    if n > 5:
        n = 5
    return df[column].str.replace('[^a-zA-Z,]', '').str.split(",").apply(lambda x: list(x)[0:n])\
                                                                .apply(lambda x:x.count(result))

# count wins, losses and draws from last 4 matches
n = 4

df["wins_home"] = count_perform(df, "aproveitamento_mandante", "v", n)
df["losses_home"] = count_perform(df, "aproveitamento_mandante", "d", n)
df["draws_home"] = count_perform(df, "aproveitamento_mandante", "e", n)

df["wins_away"] = count_perform(df, "aproveitamento_visitante", "v", n)
df["losses_away"] = count_perform(df, "aproveitamento_visitante", "d", n)
df["draws_away"] = count_perform(df, "aproveitamento_visitante", "e", n)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 19 columns):
aproveitamento_mandante     280 non-null object
aproveitamento_visitante    280 non-null object
clube_casa_id               280 non-null int64
clube_casa_posicao          280 non-null int64
clube_visitante_id          280 non-null int64
clube_visitante_posicao     280 non-null int64
local                       280 non-null object
partida_data                280 non-null object
partida_id                  280 non-null int64
placar_oficial_mandante     277 non-null float64
placar_oficial_visitante    277 non-null float64
rodada_id                   280 non-null int64
valida                      280 non-null bool
wins_home                   280 non-null int64
losses_home                 280 non-null int64
draws_home                  280 non-null int64
wins_away                   280 non-null int64
losses_away                 280 non-null int64
draws_away                  280 non-null i

In [506]:
# select and rename columns from a dataset. And create a colum to flag home teams
def div_rounds(df, cols_dict, home_flag):
    df_ret = df[list(cols_dict.keys())].rename(columns=cols_dict)
    df_ret["home"] = home_flag
    return df_ret

# filter home teams records from the rounds
cols_dict = {"clube_casa_id":"clube_id", "aproveitamento_mandante":"aproveitamento", "clube_casa_posicao":"posicao",
             "clube_casa_posicao":"in_pos", "clube_visitante_posicao":"in_pos_adv", "local":"local", "partida_data":"partida_data", 
             "partida_id":"partida_id", "placar_oficial_mandante":"placar", "placar_oficial_visitante":"placar_adv", 
             "rodada_id":"rodada_id", "valida":"valida", "wins_home":"wins", "losses_home":"losses", 
             "draws_home":"draws", "wins_away":"wins_adv", "losses_away":"losses_adv", "draws_away":"draws_adv"}

df_home = div_rounds(df, cols_dict, 1)
df_home.head()

,clube_id,aproveitamento,in_pos,in_pos_adv,local,partida_data,partida_id,placar,placar_adv,rodada_id,valida,wins,losses,draws,wins_adv,losses_adv,draws_adv,home
0,284,"['', '', '', '', 'd']",13,9,Arena do Grêmio,2019-04-28 11:00:00,232414,1.0,2.0,1,True,0,0,0,0,0,0,1
1,282,"['', '', '', '', 'v']",8,12,Independência,2019-04-27 19:00:00,232415,2.0,1.0,1,True,0,0,0,0,0,0,1
2,354,"['', '', '', '', 'v']",1,20,Castelão (CE),2019-04-28 16:00:00,232416,4.0,0.0,1,True,0,0,0,0,0,0,1
3,275,"['', '', '', '', 'v']",2,19,Arena Palmeiras,2019-04-28 19:00:00,232417,4.0,0.0,1,True,0,0,0,0,0,0,1
4,276,"['', '', '', '', 'v']",5,17,Morumbi,2019-04-27 16:00:00,232418,2.0,0.0,1,True,0,0,0,0,0,0,1


In [507]:
# filter away teams records from the rounds
cols_dict = {"clube_visitante_id":"clube_id", "aproveitamento_visitante":"aproveitamento", 
             "clube_visitante_posicao":"posicao",
             "clube_visitante_posicao":"in_pos", "clube_casa_posicao":"in_pos_adv", "local":"local", "partida_data":"partida_data", 
             "partida_id":"partida_id", "placar_oficial_visitante":"placar", "placar_oficial_mandante":"placar_adv", 
             "rodada_id":"rodada_id", "valida":"valida", "wins_away":"wins", "losses_away":"losses", 
             "draws_away":"draws", "wins_home":"wins_adv", "losses_home":"losses_adv", "draws_home":"draws_adv"}


df_away = div_rounds(df, cols_dict, 0)

# append the 2 dataframes of rounds home and away
df_rounds = df_home.append(df_away).sort_values("rodada_id")

df_rounds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 560 entries, 0 to 279
Data columns (total 18 columns):
clube_id          560 non-null int64
aproveitamento    560 non-null object
in_pos            560 non-null int64
in_pos_adv        560 non-null int64
local             560 non-null object
partida_data      560 non-null object
partida_id        560 non-null int64
placar            554 non-null float64
placar_adv        554 non-null float64
rodada_id         560 non-null int64
valida            560 non-null bool
wins              560 non-null int64
losses            560 non-null int64
draws             560 non-null int64
wins_adv          560 non-null int64
losses_adv        560 non-null int64
draws_adv         560 non-null int64
home              560 non-null int64
dtypes: bool(1), float64(2), int64(12), object(3)
memory usage: 79.3+ KB


In [508]:
# shift column in_pos by 1 round to use it as a feature
df_shiftpos = pd.DataFrame(df_rounds)
df_shiftpos = df_shiftpos.sort_values("rodada_id")
df_shiftpos["in_pos"] = df_shiftpos.groupby("clube_id")["in_pos"].shift(periods=1)

# adjust in_pos for home and away teams
df_shiftpos_away = df_shiftpos[df_shiftpos["home"]==0][["in_pos"]]
df_shiftpos_home = df_shiftpos[df_shiftpos["home"]==1][["in_pos"]]

df_merged = df_shiftpos_away.join(df_shiftpos_home, lsuffix="_adv")
df_shifted = df_merged.join(df_rounds, lsuffix="_shifted")

# replace in_pos and in_pos_adv with shifted position 
s_pos = df_shifted.apply(lambda row: [row["in_pos_adv_shifted"], row["in_pos_shifted"]] if (row["home"] == 0) else [row["in_pos_shifted"], row["in_pos_adv_shifted"]], axis=1)
df_shifted["in_pos"] = s_pos.apply(lambda x: x[0])
df_shifted["in_pos_adv"] = s_pos.apply(lambda x: x[1])

# drop temp columns used to shift
df_rounds = df_shifted.drop(columns=["in_pos_adv_shifted", "in_pos_shifted"])
df_rounds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 560 entries, 0 to 279
Data columns (total 18 columns):
clube_id          560 non-null int64
aproveitamento    560 non-null object
in_pos            540 non-null float64
in_pos_adv        540 non-null float64
local             560 non-null object
partida_data      560 non-null object
partida_id        560 non-null int64
placar            554 non-null float64
placar_adv        554 non-null float64
rodada_id         560 non-null int64
valida            560 non-null bool
wins              560 non-null int64
losses            560 non-null int64
draws             560 non-null int64
wins_adv          560 non-null int64
losses_adv        560 non-null int64
draws_adv         560 non-null int64
home              560 non-null int64
dtypes: bool(1), float64(4), int64(10), object(3)
memory usage: 79.3+ KB


In [509]:
# extract the match result from field "aproveitamento"
df_rounds["result"] = df_rounds["aproveitamento"].str.replace('[^a-zA-Z,]', '').str.split(",")\
                        .apply(lambda x: list(x)[4])

df_rounds.head()

,clube_id,aproveitamento,in_pos,in_pos_adv,local,partida_data,partida_id,placar,placar_adv,rodada_id,valida,wins,losses,draws,wins_adv,losses_adv,draws_adv,home,result
0,284,"['', '', '', '', 'd']",NaN,NaN,Arena do Grêmio,2019-04-28 11:00:00,232414,1.0,2.0,1,True,0,0,0,0,0,0,1,d
0,277,"['', '', '', '', 'v']",NaN,NaN,Arena do Grêmio,2019-04-28 11:00:00,232414,2.0,1.0,1,True,0,0,0,0,0,0,0,v
1,314,"['', '', '', '', 'd']",NaN,NaN,Independência,2019-04-27 19:00:00,232415,1.0,2.0,1,True,0,0,0,0,0,0,0,d
1,282,"['', '', '', '', 'v']",NaN,NaN,Independência,2019-04-27 19:00:00,232415,2.0,1.0,1,True,0,0,0,0,0,0,1,v
2,341,"['', '', '', '', 'd']",NaN,NaN,Castelão (CE),2019-04-28 16:00:00,232416,0.0,4.0,1,True,0,0,0,0,0,0,0,d


In [510]:
# convert datetime column
# df_rounds["match_date"] = pd.to_datetime(df_rounds["partida_data"])

# on first round teams dont have a position defined, then use 0 when NaN
df_rounds["in_pos"].fillna(value=0, inplace=True)
df_rounds["in_pos_adv"].fillna(value=0, inplace=True)

# remove samples when there is no score recorded
df_rounds = df_rounds.dropna(subset=["placar", "placar_adv"])

# remove samples not used to the game anda drop the column "valida"
df_rounds = df_rounds[df_rounds["valida"]==1]
df_rounds = df_rounds.drop(columns=["valida"])

df_rounds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 552 entries, 0 to 279
Data columns (total 18 columns):
clube_id          552 non-null int64
aproveitamento    552 non-null object
in_pos            552 non-null float64
in_pos_adv        552 non-null float64
local             552 non-null object
partida_data      552 non-null object
partida_id        552 non-null int64
placar            552 non-null float64
placar_adv        552 non-null float64
rodada_id         552 non-null int64
wins              552 non-null int64
losses            552 non-null int64
draws             552 non-null int64
wins_adv          552 non-null int64
losses_adv        552 non-null int64
draws_adv         552 non-null int64
home              552 non-null int64
result            552 non-null object
dtypes: float64(4), int64(10), object(4)
memory usage: 81.9+ KB


In [512]:
preproc_path_file = "../../data/rounds_preprocessed.csv"

df_rounds.to_csv(preproc_path_file, encoding='utf_16')

